# ***Pipeline To Fetch Data From Google Doc***

You’ll need to create a service account in Google Cloud Console → enable Google Sheets API → download the JSON key → and share your Sheet with the service account’s email (e.g. service-account@yourproject.iam.gserviceaccount.com) with Editor permission.

In [7]:
import pyodbc
import os
import pandas as pd
from datetime import datetime, date
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
from dotenv import load_dotenv , find_dotenv # to load .env files

load_dotenv()

True

In [8]:
warehouse_servr = os.environ.get("DB1_HOST")
warehouse_user = os.environ.get("DB1_USER")
warehouse_pass = os.environ.get("DB1_PASS")
warehouse_db = os.environ.get("DB1_NAME") 

conn = pyodbc.connect(
            f"Driver={{ODBC Driver 17 for SQL Server}};"
            f"Server={warehouse_servr};"  # Warehouse server
            f"Database={warehouse_db};"  # Warehouse database
            f"UID={warehouse_user};"
            f"PWD={warehouse_pass};"
        )

warehouse_cursor = conn.cursor()

warehouse_cursor.fast_executemany = True

# Connecting through Json Key

In [9]:
# Path to your downloaded key
service_acc_key = "cool-ocean-477314-v4-c7ffb11c1359.json"

# Define the required scopes
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authorize
creds = Credentials.from_service_account_file(service_acc_key, scopes=SCOPES)
gc = gspread.authorize(creds)

In [65]:
# sheet_url = 'https://docs.google.com/spreadsheets/d/1SKgKmXeInkKkhEB4eIj0bEBMXel-fvYEnqe4n7OE1hc/edit?pli=1&gid=0#gid=0'

# sh = gc.open_by_url(sheet_url) # Stores Whole Google Sheet as an Object

***Gspreas Operations like Cursor***

* **sh.title**   # returns "Title" 
* **sh.worksheets()**  # returns a list of all sheet objects inside this file
# worksheet = sh.worksheet("Sheet1")
* **worksheet.title**  # 'Sheet1'
* **worksheet.row_count**  # total number of rows
* **worksheet.col_count**  # total number of columns
* **worksheet.get_all_records()**  # returns list of dicts for all rows

In [7]:
# sheetName = sh.worksheet('Sheet1')

# ld = sheetName.get_all_records() # List of Dictionaries
# df = pd.DataFrame(ld) # Converting List of Dictionaries to DataFrame
# df

# OR

In [15]:
# df2 = get_as_dataframe(sheetName, evaluate_formulas=True, header=0)
# df2

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5941 entries, 0 to 5940
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Wave           5941 non-null   object
 1   Update         5941 non-null   object
 2   State          5941 non-null   object
 3   Territory      5941 non-null   object
 4   Cluster        5941 non-null   object
 5   Block          5941 non-null   object
 6   Block Status   5941 non-null   object
 7   Vatika         5941 non-null   object
 8   Employee Name  5941 non-null   object
 9   Activity       5941 non-null   object
 10  Target_Date    5941 non-null   object
 11  Remarks        5941 non-null   object
 12  Identify       5941 non-null   object
 13  Expected Date  5941 non-null   object
 14  Complete_Date  5941 non-null   object
 15  bhk_block_Id   5941 non-null   object
dtypes: object(16)
memory usage: 742.8+ KB


# CSP List / CSP Master

In [10]:
csp_list_url = 'https://docs.google.com/spreadsheets/d/1rVg5FKcyeg1Df4oUdc6ak-wka_YO4e8Gxs_FBexK5LU/edit?gid=0#gid=0'
csp_sheet = gc.open_by_url(csp_list_url)

In [11]:
csp_sheet.worksheets()

[<Worksheet 'Sheet1' id:0>, <Worksheet '>9 Months CSP' id:581720717>]

# Converting into DataFrame

In [12]:
csp_df = pd.DataFrame(csp_sheet.worksheet('Sheet1').get_all_records())
csp_df

,BANK,CSPCODE,CSP Name,State,Territory,District,BLOCK,Branch,PINCODE,Code Creation Date,...,Vatika ID,Vatika Name,Status,Refund Amount,Licence_Fee_Refund_Date,Employee\nMapped,Gender,Location Type,Tenure,Productive\nStatus
0,BOB,11710778,Jitesh Agrawal,Dadra And Nagar Haveli,Virtual Terr_Dadra And Nagar Haveli,Dadra And Nagar Haveli,Silvassa,Silvassa Vapi Mail Road,396230,2022-02-03,...,,NA,Active,,,Rahul Sain,M,URBAN,3.9,Active
1,BOB,11710813,Mahla Monalikumari Gulabbhai,Dadra And Nagar Haveli,Virtual Terr_Dadra And Nagar Haveli,Dadra And Nagar Haveli,Dadarand Nagar Haveli,Kilavani,396230,2022-02-03,...,,NA,Replacement,,,Rahul Sain,M,RURAL,3.9,
2,BOB,11710694,Abhinavkumar Rushabhkumar Yagnik,Gujarat,Virtual Terr_Gujarat,Ahmadabad,Vastral,"Naroda Road, Ahmedabad",,2022-03-31,...,,NA,Replacement,,,,M,,3.7,
3,BOB,11710706,Divyaben Nepubhai Makwana,Gujarat,Virtual Terr_Gujarat,Ahmadabad,bavla,Bavla,,2022-03-31,...,,NA,Replacement,,,,F,,3.7,
4,BOB,11710652,Mohasinbhai Ishakbhai Chandi,Gujarat,Virtual Terr_Gujarat,Ahmadabad,Ghanchiwada,Meghraj,383350,2022-02-01,...,,NA,Active,,,Rahul Sain,M,URBAN,3.9,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4400,SBI,10022740,,Assam,,Cachar,,,,,...,,,Closed,,,,,,,
4401,SBI,10022769,,Assam,,Cachar,,,,,...,,,Closed,,,,,,,
4402,SBI,10022881,,Maharashtra,,Ahmednagar,,,,,...,,,Closed,,,,,,,
4403,BOM,12320010,Rohit Kumar Ramani,Jharkhand,,Deoghar,,,,2025-12-15,...,,,Active,,,,,,0.0,


In [13]:
warehouse_cursor.execute("SELECT * FROM WAVE..CSP_Master")

rows_tuple = [tuple(i) for i in warehouse_cursor.fetchall()]

column = [column[0] for column in warehouse_cursor.description]

csp_master_df = pd.DataFrame(rows_tuple, columns=column)
csp_master_df

,BANK,CSPCODE,CSP_Name,Key,State,Territory,District,BLOCK,bhk_block_code,Status,...,IIBF_Certificate_Number,Printer,MATM,PinPad,Licence_Fee_Amount,MR_Date,MR_No,Received_Amount,Vatika_ID,Vatika_Name
0,BOB,11710778,Jitesh Agrawal,Dadra And Nagar Haveli-Dadra And Nagar Haveli-...,Dadra And Nagar Haveli,Virtual Terr_Dadra And Nagar Haveli,Dadra And Nagar Haveli,Silvassa,0,Active,...,,NO,0,0,0E-10,2000-01-01,0,0E-10,0,NA
1,BOB,11710813,Mahla Monalikumari Gulabbhai,Dadra And Nagar Haveli-Dadra And Nagar Haveli-...,Dadra And Nagar Haveli,Virtual Terr_Dadra And Nagar Haveli,Dadra And Nagar Haveli,Dadarand Nagar Haveli,0,Replacement,...,,NO,0,0,0E-10,2000-01-01,0,0E-10,0,NA
2,BOB,11710694,Abhinavkumar Rushabhkumar Yagnik,Gujarat-Ahmadabad-Vastral,Gujarat,Virtual Terr_Gujarat,Ahmadabad,Vastral,0,Replacement,...,,NO,0,0,0E-10,2000-01-01,0,0E-10,0,NA
3,BOB,11710706,Divyaben Nepubhai Makwana,Gujarat-Ahmadabad-bavla,Gujarat,Virtual Terr_Gujarat,Ahmadabad,bavla,4051,Replacement,...,,NO,0,0,0E-10,2000-01-01,0,0E-10,0,NA
4,BOB,11710652,Mohasinbhai Ishakbhai Chandi,Gujarat-Ahmadabad-Ghanchiwada,Gujarat,Virtual Terr_Gujarat,Ahmadabad,Ghanchiwada,0,Active,...,801852292,NO,0,0,0E-10,2000-01-01,0,0E-10,0,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5096,SBI,10022740,,,Assam,,Cachar,,0,Closed,...,,NO,0,0,0E-10,2000-01-01,0,0E-10,0,
5097,SBI,10022769,,,Assam,,Cachar,,0,Closed,...,,NO,0,0,0E-10,2000-01-01,0,0E-10,0,
5098,SBI,10022881,,,Maharashtra,,Ahmednagar,,0,Closed,...,,NO,0,0,0E-10,2000-01-01,0,0E-10,0,
5099,BOM,12320010,Rohit Kumar Ramani,,Jharkhand,,Deoghar,,0,Active,...,,NO,0,0,11800.0000000000,2025-12-12,8115,11800.0000000000,0,


# Transformation

In [14]:
csp_df.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['BANK', 'CSPCODE', 'CSP Name', 'State', 'Territory', 'District',
       'BLOCK', 'Branch', 'PINCODE', 'Code Creation Date', 'Agreement Date',
       'Agreement Renewal Date', 'IIBF Certificate\nNumber', 'Printer', 'MATM',
       'PinPad', 'Licence Fee\n Amount', 'MR Date', 'MR No.',
       'Received Amount', 'Vatika ID', 'Vatika Name', 'Status',
       'Refund Amount', 'Licence_Fee_Refund_Date', 'Employee\nMapped',
       'Gender', 'Location Type', 'Tenure', 'Productive\nStatus'],
      dtype='object')>

# Replacing Null Values

In [15]:
csp_df['PINCODE'] = pd.to_numeric(csp_df['PINCODE'], errors='coerce').fillna(0).astype('int')
csp_df['MATM'] = pd.to_numeric(csp_df['MATM'], errors='coerce').fillna(0).astype('int')
csp_df['PinPad'] = pd.to_numeric(csp_df['PinPad'], errors='coerce').fillna(0).astype('int')
csp_df['Vatika ID'] = pd.to_numeric(csp_df['Vatika ID'], errors='coerce').fillna(0).astype('int')
csp_df['MR No.'] = pd.to_numeric(csp_df['MR No.'], errors='coerce').fillna(0).astype('int')
csp_df['Licence Fee\n Amount'] = pd.to_numeric(csp_df['Licence Fee\n Amount'], errors='coerce').fillna(0.0)
csp_df['Received Amount'] = pd.to_numeric(csp_df['Received Amount'], errors='coerce').fillna(0.0)

In [16]:
#  'Vatika Name', 'Status',
#        'Refund Amount', 'Licence_Fee_Refund_Date', 'Employee\nMapped',
#        'Gender', 'Location Type']
fill_values = {
    'BANK': "",
    'CSPCODE': "",
    'CSP Name': "",
    'Key': "",
    'State': "",
    'Territory': "",
    'District': "",
    'BLOCK': "",
    'Status': "",
    'Branch': "",
    'Code Creation Date': '1999-01-01',        # keep datetime Null
    'Agreement Date': '1999-01-01',
    'Agreement Renewal Date': '1999-01-01',
    'IIBF Certificate\nNumber': "",
    'Printer': "",
    'MR Date': '1999-01-01',
    'Vatika Name': ""
}

for col, default_value in fill_values.items():
    if col in csp_df.columns:
        csp_df[col] = csp_df[col].fillna(default_value)

# Correcting Datatypes

In [17]:
dtype_map = {
    'BANK': 'object',
    'CSPCODE': 'object',
    'CSP Name': 'object',
    'Key': 'object',
    'State': 'object',
    'Territory': 'object',
    'District': 'object',
    'BLOCK': 'object',
    'Status': 'object',
    'Branch': 'object',
    'PINCODE': 'int',
    'Code Creation Date': 'datetime64[ns]',
    'Agreement Date': 'datetime64[ns]',
    'Agreement Renewal Date': 'datetime64[ns]',
    'IIBF Certificate\nNumber': 'object',
    'Printer': 'object',
    'MATM': 'int',
    'PinPad': 'int',
    'Licence Fee\n Amount': 'float',
    'MR Date': 'datetime64[ns]',
    'MR No': 'int',
    'Received Amount': 'float',
    'Vatika ID': 'int',
    'Vatika Name': 'object'
}

# Apply safe conversions
for col, new_type in dtype_map.items():
    if col in csp_df.columns:
        try:
            if "datetime" in new_type:
                csp_df[col] = pd.to_datetime(csp_df[col], errors='coerce')
            else:
                csp_df[col] = csp_df[col].astype(new_type)
        except Exception as e:
            print(f"Datatype conversion failed for {col}: {e}")

In [18]:
csp_df['Code Creation Date'] = pd.to_datetime(csp_df['Code Creation Date'])
csp_df['Agreement Date'] = pd.to_datetime(csp_df['Agreement Date'])
csp_df['Agreement Renewal Date'] = pd.to_datetime(csp_df['Agreement Renewal Date'])
# csp_df['Licence Fee Refund Date'] = pd.to_datetime(csp_df['Licence Fee Refund Date'])
csp_df['MR Date'] = pd.to_datetime(csp_df['MR Date'])
# csp_df['Licence Fee Amount'] = csp_df['Licence Fee Amount'].astype('float')
csp_df['Received Amount'] = csp_df['Received Amount'].astype('float') 
csp_df['Vatika ID'] = csp_df['Vatika ID'].astype('int')

In [19]:
# [f"{i} - {csp_df[i].dtype}" for i in csp_df.columns]
# print([f"{i} - {csp_master_df[i].dtype}" for i in csp_master_df.columns])

In [20]:
csp_df['CSPCODE'] = csp_df['CSPCODE'].astype(str).str.strip()
csp_master_df['CSPCODE'] = csp_master_df['CSPCODE'].astype(str).str.strip()

# Removing Cancelled & TBA from csp_master

In [21]:
csp_master_df_1 = csp_master_df[~csp_master_df['CSPCODE'].isin(['Cancelled', 'TBA'])]

In [16]:
csp_master_df_1.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['BANK', 'CSPCODE', 'CSP_Name', 'Key', 'State', 'Territory', 'District',
       'BLOCK', 'bhk_block_code', 'Status', 'Branch', 'PINCODE',
       'Code_Creation_Date', 'Agreement_Date', 'Agreement_Renewal_Date',
       'IIBF_Certificate_Number', 'Printer', 'MATM', 'PinPad',
       'Licence_Fee_Amount', 'MR_Date', 'MR_No', 'Received_Amount',
       'Vatika_ID', 'Vatika_Name'],
      dtype='object')>

In [17]:
# set(csp_df.columns) - set(csp_master_df.columns) 

In [22]:
csp_1 = pd.merge(csp_df, csp_master_df_1, left_on='CSPCODE', right_on = 'CSPCODE', how='left')

# csp_1 = csp_df.merge(csp_master_df_1,on="CSPCODE",how="left",suffixes=("", "_master"))

In [163]:
# csp_1['CSPCODE'].count()
more_than_1 = csp_1.groupby(['CSPCODE'])['CSPCODE'].agg(['count']).sort_values('count', ascending=False).reset_index()[csp_1.groupby(['CSPCODE'])['CSPCODE'].agg(['count']).sort_values('count', ascending=False).reset_index()['count'] > 1]

In [36]:
more_than_1

,CSPCODE,count
0,Cancelled,2209
1,TBA,784
2,10022394 (R),2
3,10022485 (R),2
4,10022267 (R),2
5,10022005 (R),2
6,10022591 (R),2
7,10022206 (R),2
8,K3800744,2
9,10022131 (R),2


In [95]:
# csp_1.columns.to_list

In [96]:
# csp_master_df.columns.to_list

In [23]:
csp_1 = csp_1[['BANK_x', 'CSPCODE', 'CSP Name', 'State_x', 'Territory_x', 'District_x',
       'BLOCK_x', 'bhk_block_code', 'Status_x', 'Branch_x', 'PINCODE_x', 'Code Creation Date',
       'Agreement Date', 'Agreement Renewal Date', 'IIBF Certificate\nNumber',
       'Printer_x', 'MATM_x', 'PinPad_x', 'Licence Fee\n Amount', 'MR Date',
       'MR No.', 'Received Amount', 'Vatika ID', 'Vatika Name','Key']]

# Renaming Columns

In [24]:
csp_1 = csp_1.rename(columns={
    'BANK_x' : 'BANK',
    'CSP Name': 'CSP_Name',
    'State_x': 'State',
    'Territory_x': 'Territory',
    'District_x': 'District',
    'BLOCK_x': 'BLOCK',
    'Branch_x': 'Branch',
    'PINCODE_x': 'PINCODE',
    'Code Creation Date': 'Code_Creation_Date',
    'Agreement Date': 'Agreement_Date',
    'Agreement Renewal Date': 'Agreement_Renewal_Date',
    'IIBF Certificate\nNumber': 'IIBF_Certificate_Number',
    'Printer_x': 'Printer',
    'MATM_x': 'MATM',
    'PinPad_x': 'PinPad',
    'Licence Fee\n Amount': 'Licence_Fee_Amount',
    'MR Date': 'MR_Date',
    'MR No.': 'MR_No',
    'Received Amount': 'Received_Amount',
    'Vatika ID': 'Vatika_ID',
    'Vatika Name': 'Vatika_Name',
    'Status_x': 'Status'
    # Any extra columns (Refund Amount, Employee Mapped etc.) will remain
})

In [25]:
csp_1['bhk_block_code'] = csp_1['bhk_block_code'].fillna(0)
csp_1['bhk_block_code'] = csp_1['bhk_block_code'].astype(int)
csp_1['Code_Creation_Date'].fillna('2000-01-01', inplace=True)
csp_1['Agreement_Date'].fillna('2000-01-01', inplace=True)
csp_1['Agreement_Renewal_Date'].fillna('2000-01-01', inplace=True)
csp_1['MR_Date'].fillna('2000-01-01', inplace=True)
csp_1['Key'].fillna('', inplace=True)

C:\Users\handa\AppData\Local\Temp\ipykernel_3444\2594370257.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  csp_1['Code_Creation_Date'].fillna('2000-01-01', inplace=True)
C:\Users\handa\AppData\Local\Temp\ipykernel_3444\2594370257.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [26]:
warehouse_cursor.execute("TRUNCATE TABLE WAVE..CSP_Master")

In [ ]:


warehouse_cursor.execute("SELECT * FROM WAVE..CSP_Master")
rows_tuple = [tuple(i) for i in warehouse_cursor.fetchall()]
column = [column[0] for column in warehouse_cursor.description]
csp_master_df_check = pd.DataFrame(rows_tuple, columns=column)

if len(csp_master_df_check) == 0:
    print(f"CSP_Master Truncated...! {len(csp_master_df_check)} Records Found")


0


In [28]:
table_name = "CSP_Master"  # Replace with your actual table name
batch_size = 1000
idf = csp_1

try:
    columns = idf.columns.tolist()
    # columns_str = ', '.join(columns)
    columns_str = ', '.join([f'[{col}]' for col in columns])
    placeholders = ', '.join(['?' for _ in range(len(columns))])
    insert_query = f"INSERT INTO WAVE..{table_name} ({columns_str}) VALUES ({placeholders})"
    
    total_inserted = 0
    
    # Process in batches
    for i in range(0, len(idf), batch_size):
        batch = idf.iloc[i: i + batch_size]
        records = [tuple(row) for _, row in batch.iterrows()]
        
        print(f"Executing batch insert for records {i} to {i + len(records) - 1}")
        warehouse_cursor.executemany(insert_query, records)
        
        total_inserted += len(records) 
        print(f"Inserted batch: {total_inserted}/{len(idf)} records")
              
    
    print(f"✓ Successfully inserted all {total_inserted} records into {table_name}")
    conn.commit()
    conn.close()
    
except Exception as e:
    # conn.rollback()
    print(f"✗ Error inserting data: {str(e)}")
    
# finally:
#     warehouse_cursor.close()
#     conn.close()

Executing batch insert for records 0 to 999
Inserted batch: 1000/4471 records
Executing batch insert for records 1000 to 1999
Inserted batch: 2000/4471 records
Executing batch insert for records 2000 to 2999
Inserted batch: 3000/4471 records
Executing batch insert for records 3000 to 3999
Inserted batch: 4000/4471 records
Executing batch insert for records 4000 to 4470
Inserted batch: 4471/4471 records
✓ Successfully inserted all 4471 records into CSP_Master


In [63]:
# warehouse_cursor.execute("TRUNCATE TABLE WAVE..CSP_Master_tmp")
conn.commit()
conn.close()